In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
%matplotlib inline

In [2]:
%cd /content/drive/MyDrive/PHI/ToF_ML/src

/content/drive/MyDrive/PHI/ToF_ML/src


In [3]:
%load_ext tensorboard
import datetime, os
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [4]:
from ast import literal_eval
data = pd.read_csv('../data/fixed_1400.csv')
data['masses'] = data['masses'].apply(literal_eval)
data['channels'] = data['channels'].apply(literal_eval)
data['intensities'] = data['intensities'].apply(literal_eval)

In [5]:
data.head()

,file_name,Mass/Time,MassOffset,StartFlightTime,SpecBinSize,channels,intensities,masses,avg_dist_frags_low,avg_dist_frags_high,adjusted_original_proportions_identified,original_proportions_identified,diff,prop_diff_in_low,calibration,adjusted_proportion_identified,proportion_identified
0,0106301.cas,0.387384,-0.275004,0.0,1.248,"[2644.0367300000003, 3505.0183700000002, 4162....","[73874, 1234, 138, 610, 1216, 4159, 8958, 1084...","[1.0065519723918102, 2.015029094672708, 3.0191...",0.001298,0.002255,0.418033,0.398438,0.000958,0.738174,0,0.540984,0.515625
1,0107316.cas,0.387113,-0.278302,0.0,1.248,"[2647.00072, 3508.9949100000003, 4164.59326000...","[49864, 1034, 168, 4696, 8247, 13992, 17903, 2...","[1.00101811517532, 2.0077555328930656, 3.00565...",0.001537,0.002586,0.131783,0.129771,0.001049,0.682225,0,0.519380,0.511450
2,0110203.cas,0.379037,-0.271056,0.0,4.992,"[1973.87665, 2049.0706800000003, 2122.01224, 2...","[23352, 74717, 10387, 947, 12344, 9121, 249, 4...","[11.998071176139083, 13.003971096434277, 14.01...",0.001640,0.001858,0.388889,0.388889,0.000218,0.133100,0,0.444444,0.444444
3,0110212.cas,0.379177,-0.269744,0.0,4.992,"[672.00298, 891.90543, 1970.94521, 2046.11295,...","[34398, 304, 2223, 3521, 5205, 6509, 99, 115, ...","[1.0045194511091773, 2.012140966655108, 11.978...",0.001337,0.002206,0.379630,0.379630,0.000868,0.649178,0,0.592593,0.592593
4,0116511.cas,0.383360,-0.302184,0.0,1.248,"[2726.98153, 3594.53649, 4265.28736, 7866.5038...","[42995, 602, 151, 17912, 9536, 17609, 29604, 5...","[1.0049940659400325, 2.0094784706009245, 3.022...",0.001397,0.002171,0.418367,0.336066,0.000774,0.554114,0,0.581633,0.467213


In [6]:
from data_transformation import generate_data, mass_formula, generate_calibrated_data
erred = generate_data(data, 2, 2, [0, 0, 0], slope_index=2)
for num in range(3):
    for _ in range(10):
        erred = pd.concat([erred, generate_data(data, num + 2, 1, [0.25, .5, 1], True, True, slope_index=2)], axis=0)
erred.reset_index(inplace=True, drop=True)
erred = generate_calibrated_data(erred, slope_index=2)

In [7]:
erred['target'].describe()

count    44671.000000
mean         1.455821
std          0.559081
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          2.000000
Name: target, dtype: float64

In [8]:
from data_transformation import get_frags

In [9]:
frags = get_frags()['FragmentMass']

In [10]:
def get_frag_dists(masses, frags, thresh=0.003, ab=True):
    '''
    Determines which elemental / compound masses correspond
    to actual spectra masses and returns both the fragments
    and the distance between each fragment and its related mass in
    the given spectra.

    Arguments -------
    masses: list of masses for a spectrum
    frags: fragment list
    thresh: how close a fragment must be to a peak for it to be matched,
            default 3 mamu
    ab: whether to use absolute value for calculated distances, affects
        the average distance per spectrum.
    '''
    found_masses = []
    found_frags = []
    dists = []
    for mass in masses:
        not_found = True
        i = (len(frags)) // 2
        floor = 0
        cieling = len(frags) - 1
        min = 1

        def is_findable():
            if abs(floor - cieling) <= 1:
                dists.append(min)
                return False
            return True
        
        while not_found:
            dist = frags[i] - mass
            if dist < min:
                min = dist
            if abs(dist) < thresh:
                not_found = False
                i = get_closest(i, frags, mass)
                if ab:
                    dists.append(abs(frags[i] - mass))
                else:
                    dists.append((frags[i] - mass))
            elif dist > 0:
                not_found = is_findable()
                cieling = i
                num = abs(floor - i)
                if num != 1:
                    i -= abs(floor - i) // 2
                else:
                    i -= 1
            else:
                not_found = is_findable()
                floor = i
                num = abs(cieling - i)
                if num != 1:
                    i += abs(cieling - i) // 2
                else:
                    i += 1
    return dists

In [11]:
def get_closest(i, frags, mass):
    '''
    Recursively checks that the closest fragment to a peak is selected.

    Arguments ------
    i: index in fragment list to start checking
    frags: list of mass fragments
    mass: mass of peak being matched
    '''
    d = abs(frags[i] - mass)
    if len(frags) > i + 1 and d > abs(frags[i + 1] - mass):
        i = get_closest(i + 1, frags, mass)
    elif i - 1 >= 0 and d > abs(frags[i - 1] - mass):
        i = get_closest(i - 1, frags, mass)
    return i

In [12]:
def get_spectra_summary(masses, scaled_intens, slope, offset, frags, length=2000):
    # scaled intens might not be a good tie breaker for peaks at same mass
    summary = np.zeros([3, length + 1])
    i = 0
    distances = get_frag_dists(masses, frags)
    while 1:
        mass = masses[i]
        if mass < length:
            intensity = scaled_intens[i]
            distance = distances[i]
            index = round(mass)
            if summary[1][index] == 0:
                summary[2][index] = distance
                summary[1][index] = mass - round(mass)
                summary[0][index] = intensity
            else:
                if intensity > summary[0][index]:
                    summary[2][index] = distance
                    summary[1][index] = mass - round(mass)
                    summary[0][index] = intensity
        i += 1
        if i >= len(masses):
            break
    summary[0][-1] += slope
    summary[1][-1] += offset
    return summary

In [13]:
from sklearn.preprocessing import MinMaxScaler
scl = MinMaxScaler()
scaled_intens = []
for row in erred.itertuples():
    scl.fit(np.array(row.intensities).reshape((-1, 1)))
    intensities = scl.transform(np.array(row.intensities).reshape((-1, 1)))
    scaled_intens.append(intensities)

In [14]:
X = np.zeros((len(erred), 3, 1001))
y = np.array(erred['target'])
for i, row in enumerate(erred.itertuples()):
    print('row: ', i)
    summary = get_spectra_summary(row.masses, scaled_intens[i], row[2],
                                  row.MassOffset, frags, length=1000)
    X[i] += summary

Streaming output truncated to the last 5000 lines.
row:  39671
row:  39672
row:  39673
row:  39674
row:  39675
row:  39676
row:  39677
row:  39678
row:  39679
row:  39680
row:  39681
row:  39682
row:  39683
row:  39684
row:  39685
row:  39686
row:  39687
row:  39688
row:  39689
row:  39690
row:  39691
row:  39692
row:  39693
row:  39694
row:  39695
row:  39696
row:  39697
row:  39698
row:  39699
row:  39700
row:  39701
row:  39702
row:  39703
row:  39704
row:  39705
row:  39706
row:  39707
row:  39708
row:  39709
row:  39710
row:  39711
row:  39712
row:  39713
row:  39714
row:  39715
row:  39716
row:  39717
row:  39718
row:  39719
row:  39720
row:  39721
row:  39722
row:  39723
row:  39724
row:  39725
row:  39726
row:  39727
row:  39728
row:  39729
row:  39730
row:  39731
row:  39732
row:  39733
row:  39734
row:  39735
row:  39736
row:  39737
row:  39738
row:  39739
row:  39740
row:  39741
row:  39742
row:  39743
row:  39744
row:  39745
row:  39746
row:  39747
row:  39748
row:  39749
r

In [15]:
from sklearn.model_selection import train_test_split
indices = np.concatenate((np.random.randint(0, 1441, 10), np.random.randint(1441, X.shape[0], 270)))
X_val = X[indices]
y_val = y[indices]
X = np.delete(X, indices, axis=0)
y = np.delete(y, indices, axis=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
X_val = tf.convert_to_tensor(X_val)

In [17]:
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [39]:
class_weights = {0: 1 / (np.sum(erred['target']==0) / len(erred)), 1: 1/ (np.sum(erred['target']==1) / len(erred)), 2: 1/ (np.sum(erred['target']==2))}

In [40]:
class_weights

{0: 31.0, 1: 2.0847995519671443, 2: 4.586524790166491e-05}

In [20]:
import tensorflow.keras as keras

In [42]:
def calculate_metrics(y_true, y_pred, duration):
    res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall', 'duration'])
    res['precision'] = precision_score(y_true, y_pred, average='macro')
    res['accuracy'] = accuracy_score(y_true, y_pred)
    res['recall'] = recall_score(y_true, y_pred, average='macro')
    res['duration'] = duration
    return res


def save_test_duration(file_name, test_duration):
    res = pd.DataFrame(data=np.zeros((1, 1), dtype=np.float), index=[0],
                       columns=['test_duration'])
    res['test_duration'] = test_duration
    res.to_csv(file_name, index=False)


def save_logs(output_directory, hist, y_pred, y_true, duration,
              lr=True, plot_test_acc=True):
    hist_df = pd.DataFrame(hist.history)
    hist_df.to_csv(output_directory + 'history.csv', index=False)

    df_metrics = calculate_metrics(y_true, y_pred, duration)
    df_metrics.to_csv(output_directory + 'df_metrics.csv', index=False)

    index_best_model = hist_df['loss'].idxmin()
    row_best_model = hist_df.loc[index_best_model]

    df_best_model = pd.DataFrame(data=np.zeros((1, 6), dtype=np.float),
                                 index=[0],
                                 columns=['best_model_train_loss',
                                          'best_model_val_loss',
                                          'best_model_train_acc',
                                          'best_model_val_acc',
                                          'best_model_learning_rate',
                                          'best_model_nb_epoch'])
    df_best_model['best_model_train_loss'] = row_best_model['loss']
    if plot_test_acc:
        df_best_model['best_model_val_loss'] = row_best_model['val_loss']
    df_best_model['best_model_train_acc'] = row_best_model['accuracy']
    if plot_test_acc:
        df_best_model['best_model_val_acc'] = row_best_model['val_accuracy']
    if lr == True:
        df_best_model['best_model_learning_rate'] = row_best_model['lr']
    df_best_model['best_model_nb_epoch'] = index_best_model

    df_best_model.to_csv(output_directory + 'df_best_model.csv', index=False)

    if plot_test_acc:
        # plot losses
        plot_epochs_metric(hist, output_directory + 'epochs_loss.png')

    return df_metrics


def plot_epochs_metric(hist, file_name, metric='loss'):
    plt.figure()
    plt.plot(hist.history[metric])
    plt.plot(hist.history['val_' + metric])
    plt.title('model ' + metric)
    plt.ylabel(metric, fontsize='large')
    plt.xlabel('epoch', fontsize='large')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(file_name, bbox_inches='tight')
    plt.close()



class Classifier_INCEPTION:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, build=True, batch_size=64,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs
        self.verbose = verbose

        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        input_dropout = keras.layers.Dropout(rate=0.2)(input_layer)
        x = input_dropout
        input_res = input_dropout

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        dropout_gap = keras.layers.Dropout(rate=.2)(gap_layer)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(dropout_gap)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy', 'AUC'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
                                                      min_lr=0.0001)

        file_path = self.output_directory + 'best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True, )
        
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir,
                                                              histogram_freq=1)
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                         min_delta=0,
                                         patience=50,
                                         verbose=0,
                                         mode="auto",
                                         baseline=None,
                                         restore_best_weights=False,
                                        )

        self.callbacks = [reduce_lr, model_checkpoint, tensorboard_callback,
                          early_stopping]

        return model

    def fit(self, x_train, y_train, x_val, y_val, y_true, plot_test_acc=False):
        if not tf.test.gpu_device_name():
            print('error no gpu')
            exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training

        if self.batch_size is None:
            mini_batch_size = int(min(x_train.shape[0] / 10, 16))
        else:
            mini_batch_size = self.batch_size

        start_time = time.time()

        if plot_test_acc:

            hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size,
                                  epochs=self.nb_epochs, verbose=self.verbose,
                                  validation_data=(x_val, y_val),
                                  callbacks=self.callbacks,
                                  class_weight=class_weights)
        else:

            hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size,
                                  epochs=self.nb_epochs, verbose=self.verbose,
                                  callbacks=self.callbacks,
                                  class_weight=class_weights )

        duration = time.time() - start_time

        self.model.save(self.output_directory + 'last_model.hdf5')

        y_pred = self.predict(x_val, y_true, x_train, y_train, y_val,
                              return_df_metrics=False)

        # save predictions
        np.save(self.output_directory + 'y_pred.npy', y_pred)

        # convert the predicted from binary to integer
        y_pred = np.argmax(y_pred, axis=1)

        df_metrics = save_logs(self.output_directory, hist, y_pred, y_true, duration,
                             plot_test_acc=plot_test_acc)

        keras.backend.clear_session()

        return 0#

    def predict(self, x_test, y_true, x_train, y_train, y_test, return_df_metrics=True):
        start_time = time.time()
        model_path = self.output_directory + 'best_model.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test, batch_size=self.batch_size)
        if return_df_metrics:
            y_pred = np.argmax(y_pred, axis=1)
            df_metrics = calculate_metrics(y_true, y_pred, 0.0)
            return df_metrics
        else:
            test_duration = time.time() - start_time
            save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
            return y_pred

In [31]:
X_train.shape

TensorShape([35512, 3, 1001])

In [23]:
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import LabelEncoder

In [32]:
c = Classifier_INCEPTION('../data/', (3, 1001), 3, build=True, verbose=True, batch_size=60, nb_epochs=200, depth=5, use_bottleneck=True)

In [30]:
y_train.shape

(35512,)

In [43]:
y_train_dummies = np.array(pd.get_dummies(pd.Series(np.array(y_train).reshape(y_train.shape[0], ))))
y_val_dummies =  np.array(pd.get_dummies(pd.Series(np.array(y_val).reshape(y_val.shape[0], ))))

In [44]:
c.fit(X_train, y_train_dummies, X_val, y_val_dummies, y_val, plot_test_acc=True)

Epoch 1/200
592/592 [==============================] - 10s 18ms/step - loss: 1.1870 - accuracy: 0.2401 - auc: 0.4246 - val_loss: 6.1254 - val_accuracy: 0.2536 - val_auc: 0.4189
Epoch 2/200
592/592 [==============================] - 10s 16ms/step - loss: 1.1846 - accuracy: 0.2766 - auc: 0.4326 - val_loss: 6.2304 - val_accuracy: 0.2321 - val_auc: 0.4110
Epoch 3/200
592/592 [==============================] - 10s 16ms/step - loss: 1.1776 - accuracy: 0.2642 - auc: 0.4363 - val_loss: 6.3621 - val_accuracy: 0.2000 - val_auc: 0.4208
Epoch 4/200
592/592 [==============================] - 10s 16ms/step - loss: 1.1847 - accuracy: 0.2480 - auc: 0.4304 - val_loss: 6.0901 - val_accuracy: 0.1821 - val_auc: 0.4033
Epoch 5/200
592/592 [==============================] - 10s 16ms/step - loss: 1.1870 - accuracy: 0.2490 - auc: 0.4251 - val_loss: 6.0886 - val_accuracy: 0.2321 - val_auc: 0.4237
Epoch 6/200
592/592 [==============================] - 10s 16ms/step - loss: 1.1691 - accuracy: 0.2549 - auc: 0.433

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0

In [ ]:
c.model.save('../models/scaled_intens_whole_num_subbed_masses_3_cat_dropout')

In [ ]:
 keras.models.load_model('../models/scaled_intens_whole_num_subbed_masses')

In [ ]:
c.predict(X_val, y_val, X_train, y_train, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,accuracy,recall,duration
0,0.500845,0.792857,0.544176,0.0


In [ ]:
%tensorboard --logdir log_dir

In [ ]:
from keras.utils import to_categorical
from PIL import Image
# Data Generator Example: read in data small amt at a time to save memory
# https://towardsdatascience.com/building-a-multi-output-convolutional-neural-network-with-keras-ed24c7bc1178
class UtkFaceDataGenerator():
    """
    Data generator for the UTKFace dataset. This class should be used when training our Keras multi-output model.
    """
    def __init__(self, df):
        self.df = df
        
    def generate_split_indexes(self):
        p = np.random.permutation(len(self.df))
        train_up_to = int(len(self.df) * TRAIN_TEST_SPLIT)
        train_idx = p[:train_up_to]
        test_idx = p[train_up_to:]
        train_up_to = int(train_up_to * TRAIN_TEST_SPLIT)
        train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]
        
        # converts alias to id
        self.df['gender_id'] = self.df['gender'].map(lambda gender: dataset_dict['gender_alias'][gender])
        self.df['race_id'] = self.df['race'].map(lambda race: dataset_dict['race_alias'][race])
        self.max_age = self.df['age'].max()
        
        return train_idx, valid_idx, test_idx
    
    def preprocess_image(self, img_path):
        """
        Used to perform some minor preprocessing on the image before inputting into the network.
        """
        im = Image.open(img_path)
        im = im.resize((IM_WIDTH, IM_HEIGHT))
        im = np.array(im) / 255.0
        
        return im
        
    def generate_images(self, image_idx, is_training, batch_size=16):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """
        
        # arrays to store our batched data
        images, ages, races, genders = [], [], [], []
        while True:
            for idx in image_idx:
                person = self.df.iloc[idx]
                
                age = person['age']
                race = person['race_id']
                gender = person['gender_id']
                file = person['file']
                
                im = self.preprocess_image(file)
                
                ages.append(age / self.max_age)
                races.append(to_categorical(race, len(dataset_dict['race_id'])))
                genders.append(to_categorical(gender, len(dataset_dict['gender_id'])))
                images.append(im)
                
                # yielding condition
                if len(images) >= batch_size:
                    yield np.array(images), [np.array(ages), np.array(races), np.array(genders)]
                    images, ages, races, genders = [], [], [], []
                    
            if not is_training:
                break
                
data_generator = UtkFaceDataGenerator(df)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes() 